In [ ]:
import logging
from fastapi import FastAPI, responses
import pandas as pd
import uvicorn

import sys
import os
import plotly.express as px
import plotly.graph_objects as go

# Get the path to the main folder (CWD for the notebook)
main_folder = os.getcwd()

# Construct the path to the src directory
src_path = os.path.join(main_folder, "src")

# Add the src directory to the module search path if it's not already there
if src_path not in sys.path:
    sys.path.append(src_path)

# Verify the path is added
print("Updated sys.path:", sys.path)

from src.ChannelAnalysis import ChannelAnalysis
from src.Plotting import Plotting
from typing import Callable

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

main_channel_analysis: ChannelAnalysis

if __name__ == "__main__":
    logging.info("Preparing data")
    main_channel_analysis = ChannelAnalysis()
    logging.info("Starting the analysis")

lore_channels: list[int] = [564402253293682689, 411461323994562562, 974683899336728576, 498485828188897281, 377847586440347658, 505705046713827338, 937966714040225842, 832943358300782642, 832943368316911626, 783838021526618132, 907293843345539162, 583022525194174500, 551520218149552128, 712252130907652117, 465930237172252683, 530423897926008832, 498603226237763596, 533329012966293530, 802219413469397082, 428791539460997120, 1132343261684572271, 702224223950602241, 498408363558961172, 504368120434262027, 527086342950092811, 498482633861234688, 542794654610096160, 552971529185525770, 566604750511013898, 516718270330568722, 801191262593220638, 801599633813012520, 746033726999494737, 431690609904975892, 908859122252910622, 791721697946370068, 748666907279753278, 711348485772804137, 779753475395026984, 780401938587058226, 789123366488506389, 790592273556832276, 801408562571116554, 707980670508728405, 712792965851906068, 746286058664361996, 804447562706845727, 508163705356419093, 498145374318493696, 498145303296213015, 498145430178103307, 498145244609773568, 374643216110190593, 385820765574004787, 478244149674508288, 495648708168515585, 382604708331782145, 374255882189537281, 382608831844319232, 702626520103125063, 594261308564832268, 431383675792261140, 500763899084275724, 725464629043789977, 719262766539472906, 594179065213353988, 498562444751405057, 503215298871164929, 703901421476642896, 701408347991310376, 802160808627929119, 702646302923816960, 916014707733827625, 745741564747317430, 746026409251831808, 785639001611829308, 731859604383727646, 720762867887898665, 811175347407814666] 

In [ ]:
channel_analysis: ChannelAnalysis = ChannelAnalysis(message_df=main_channel_analysis.messages_df)
channel_analysis.restrict_to_channels(lore_channels)

logging.getLogger().info(f"Plotting message count by day")
df: pd.DataFrame = channel_analysis.day_number_of_messages().copy()
creation_date_series_lore = df["created_at"].value_counts()
creation_date_series_lore = creation_date_series_lore.sort_index()

lore_channels_fig = px.histogram(creation_date_series_lore, 
                    x = creation_date_series_lore.index, 
                    y = "count",
                    title="Messages on lore channels",
                    labels={
                        "count": "Count",
                        "created_at": "Date"
                    },
                    barmode="group",
                    color_discrete_sequence=['red'],
                    nbins=len(creation_date_series_lore))
lore_channels_fig.show()

In [ ]:
all_channels: list[int] = None
channel_analysis: ChannelAnalysis = ChannelAnalysis(message_df=main_channel_analysis.messages_df)
channel_analysis.restrict_to_channels(all_channels)

logging.getLogger().info(f"Plotting message count by day")
df: pd.DataFrame = channel_analysis.day_number_of_messages().copy()
creation_date_series_all = df["created_at"].value_counts()
creation_date_series_all = creation_date_series_all.sort_index()

all_channels_fig = px.histogram(creation_date_series_all, 
                    x = creation_date_series_all.index, 
                    y = "count",
                    title="Messages on all channels",
                    labels={
                        "count": "Count",
                        "created_at": "Date"
                    },
                    barmode="group",
                    nbins=len(creation_date_series_all))
all_channels_fig.show()



In [ ]:
#both figure
# Combine the histograms
fig = go.Figure()

creation_date_series_all_normalized = creation_date_series_all.copy()
creation_date_series_lore_normalized = creation_date_series_lore.copy()
creation_date_series_all_normalized  = creation_date_series_all_normalized / creation_date_series_all_normalized.max()
creation_date_series_lore_normalized = creation_date_series_lore_normalized / creation_date_series_lore_normalized.max()

all_channels_fig_probability = px.bar(creation_date_series_all_normalized, 
                    x = creation_date_series_all_normalized.index, 
                    y = "count",
                    title="Messages on all channels",
                    labels={
                        "count": "Count",
                        "created_at": "Date"
                    },
                    barmode="group",
                    color_discrete_sequence=['blue'],
)

lore_channels_fig_probability = px.bar(creation_date_series_lore_normalized, 
                    x = creation_date_series_lore_normalized.index, 
                    y = "count",
                    title="Messages on lore channels",
                    labels={
                        "count": "Count",
                        "created_at": "Date"
                    },
                    barmode="group",
                    color_discrete_sequence=['red']
)

# Add traces from both histograms
for trace in all_channels_fig_probability.data:
    trace.opacity = 1
    fig.add_trace(trace)
for trace in lore_channels_fig_probability.data:
    trace.opacity = 0.5
    fig.add_trace(trace)

fig.update_traces(marker_line_width=0)

# Update layout
fig.update_layout(
    title="Lore channels on top of all channels",
    xaxis_title="Date",
    yaxis_title="Messages to max ratio",
    barmode='overlay',
    template="plotly",
    bargap=0,
    bargroupgap=0
)

fig.show()

In [ ]:
all_channels: list[int] = None
channel_analysis: ChannelAnalysis = ChannelAnalysis(message_df=main_channel_analysis.messages_df)
channel_analysis.restrict_to_channels(all_channels)

logging.getLogger().info(f"Plotting message count by day")
df: pd.DataFrame = channel_analysis.day_number_of_messages().copy()
creation_date_series_all_yearly = df["created_at"].value_counts()
creation_date_series_all_yearly = creation_date_series_all_yearly.sort_index()

creation_date_series_all_yearly.index = pd.to_datetime(creation_date_series_all_yearly.index).year

all_channels_fig = px.histogram(creation_date_series_all_yearly, 
                    x = creation_date_series_all_yearly.index, 
                    y = "count",
                    title="Messages on all channels yearly",
                    labels={
                        "count": "Count",
                        "created_at": "Date"
                    },
                    barmode="group")
all_channels_fig.show()


In [ ]:
channel_analysis: ChannelAnalysis = ChannelAnalysis(message_df=main_channel_analysis.messages_df)
channel_analysis.restrict_to_channels(lore_channels)

logging.getLogger().info(f"Plotting message count by day")
df: pd.DataFrame = channel_analysis.day_number_of_messages().copy()
creation_date_series_lore_yearly = df["created_at"].value_counts()
creation_date_series_lore_yearly = creation_date_series_lore_yearly.sort_index()

creation_date_series_lore_yearly.index = pd.to_datetime(creation_date_series_lore_yearly.index).year

all_channels_fig = px.histogram(creation_date_series_lore_yearly, 
                    x = creation_date_series_lore_yearly.index, 
                    y = "count",
                    title="Messages on all channels yearly",
                    labels={
                        "count": "Count",
                        "created_at": "Date"
                    },
                    color_discrete_sequence=['red'],
                    barmode="group")
all_channels_fig.show()


In [ ]:
#both figure
# Combine the histograms
fig = go.Figure()

creation_date_series_all_normalized = creation_date_series_all_yearly.copy()
creation_date_series_lore_normalized = creation_date_series_lore_yearly.copy()
creation_date_series_all_normalized  = creation_date_series_all_normalized / creation_date_series_all_normalized.max()
creation_date_series_lore_normalized = creation_date_series_lore_normalized / creation_date_series_lore_normalized.max()

all_channels_fig_probability = px.bar(creation_date_series_all_normalized, 
                    x = creation_date_series_all_normalized.index, 
                    y = "count",
                    title="Messages on all channels",
                    labels={
                        "count": "Count",
                        "created_at": "Date"
                    },
                    barmode="group",
                    color_discrete_sequence=['blue'],
)

lore_channels_fig_probability = px.bar(creation_date_series_lore_normalized, 
                    x = creation_date_series_lore_normalized.index, 
                    y = "count",
                    title="Messages on lore channels",
                    labels={
                        "count": "Count",
                        "created_at": "Date"
                    },
                    barmode="group",
                    color_discrete_sequence=['red']
)

# Add traces from both histograms
for trace in all_channels_fig_probability.data:
    trace.opacity = 1
    fig.add_trace(trace)
for trace in lore_channels_fig_probability.data:
    trace.opacity = 0.5
    fig.add_trace(trace)

fig.update_traces(marker_line_width=0)

# Update layout
fig.update_layout(
    title="Lore channels on top of all channels yearly",
    xaxis_title="Date",
    yaxis_title="Messages to max ratio",
    barmode='overlay',
    template="plotly",
    bargap=0,
    bargroupgap=0
)

fig.show()

In [ ]:
from urllib.request import urlopen
import json
with urlopen('file:///home/stefan/Downloads/features(19).geojson') as response:
    counties = json.load(response)

import pandas as pd
df = pd.read_csv("file:///home/stefan/Downloads/fimp.csv",
                   dtype={"fips": str})

import plotly.express as px

fig = px.choropleth(df, geojson=counties, locations='fips', color='unemp',
                           color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           labels={'unemp':'unemployment rate'},
                            featureidkey="properties.id"
                          )
fig.update_geos(visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
